In [0]:
def fetch_posts():
  for line in open("data.tsv", "r"):
    post_id, text = line.split("\t")
    yield int(post_id), text.strip()

In [0]:
all_answers = [q for q,v in meta.items() if v['ParentId']!=-1]
Y = np.asarray([meta[answerId]['Score']>0 for answerId in all_answers])

In [0]:
from sklearn import neighbors
knn = neighbors.KNeighborsClassifier(n_neighbors=2)
print(knn)

In [0]:
knn.fit([[1],[2],[3],[4],[5],[6]], [0,0,0,1,1,1])
print(knn.predict(1.5))
print(knn.predict(37))
print(knn.predict(3))

In [0]:
print(knn.predict_proba(1.5))
print(knn.predict_proba(37))
print(knn.predict_proba(3.5))

In [0]:
import re
code_match = re.compile('<pre>(.*?)</pre>', re.MULTILINE | re.DOTALL)
link_match = re.compile('<a href="http://.*?".*?>(.*?)</a>', re.MULTILINE | re.DOTALL)
tag_match = re.compile('<[^>]*>', re.MULTILINE | re.DOTALL)


def extract_features_from_body(s):
  link_count_in_code = 0
  for match_str in code_match.findall(s):
    link_count_in_code += len(link_match.findall(match_str))

  return len(link_match.findall(s)) - link_count_in_code

In [0]:
X = np.asarray([extract_features_from_body(text) for post_id, text in fetch_posts() if post_id in all_answers])
knn = neighbors.KNeighborsClassifier()
knn.fit(X, Y)

In [0]:
from sklearn.cross_validation import KFold
scores = []
cv = KFold(n=len(X), k=10, indices=True)
for train, test in cv:
  X_train, y_train = X[train], Y[train]
  X_test, y_test = X[test], Y[test]
  clf = neighbors.KNeighborsClassifier()
  clf.fit(X, Y)
  scores.append(clf.score(X_test, y_test))
print("Mean(scores)=%.5f\tStddev(scores)=%.5f"\ %(np.mean(scores), np.std(scores)))

In [0]:
def extract_features_from_body(s):
  num_code_lines = 0
  link_count_in_code = 0
  code_free_s = s
  for match_str in code_match.findall(s):
    num_code_lines += match_str.count('\n')
    code_free_s = code_match.sub("", code_free_s)
    link_count_in_code += len(link_match.findall(match_str))
  links = link_match.findall(s)
  link_count = len(links)
  link_count -= link_count_in_code
  html_free_s = re.sub(" +", " ", tag_match.sub('', code_free_s)).replace("\n", "")
  link_free_s = html_free_s
  for link in links:
    if link.lower().startswith("http://"):
      link_free_s = link_free_s.replace(link,'')
  num_text_tokens = html_free_s.count(" ")
  return num_text_tokens, num_code_lines, link_count

In [0]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
print(clf)

In [0]:
clf.fit(X, y)
print(np.exp(clf.intercept_), np.exp(clf.coef_.ravel()))

In [0]:
def lr_model(clf, X):
  return 1 / (1 + np.exp(-(clf.intercept_ + clf.coef_*X)))

In [0]:
print("P(x=-1)=%.2f\tP(x=7)=%.2f"%(lr_model(clf, -1), lr_model(clf, 7)))

In [0]:
from sklearn.metrics import precision_recall_curve
precision, recall, thresholds = precision_recall_curve(y_test, clf.predict(X_test))

In [0]:
medium = np.argsort(scores)[int(len(scores) / 2)]
thresholds = np.hstack(([0],thresholds[medium]))
idx80 = precisions>=0.8
print("P=%.2f R=%.2f thresh=%.2f" % (precision[idx80][0], recall[idx80][0], threshold[idx80][0]))

In [0]:
thresh80 = threshold[idx80][0]
probs_for_good = clf.predict_proba(answer_features)[:,1]
answer_class = probs_for_good>thresh80

In [0]:
from sklearn.metrics import classification_report
print(classification_report(y_test, clf.predict_proba [:,1]>0.63, target_names=['not accepted', 'accepted']))

In [0]:
import pickle
pickle.dump(clf, open("logreg.dat", "w"))
clf = pickle.load(open("logreg.dat", "r"))